## Importar Librerias

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np 
import os
from datetime import datetime 

In [2]:
#Ruta del dataset
data_path = os.path.join(os.getcwd(),'..','data_sintetica','dataset_morosidad.csv')
df = pd.read_csv(data_path)

df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,moroso
0,-0.623609,0.373243,0.151976,-0.860282,0.009626,-0.416521,0.334591,0.590542,-0.327102,1.278624,0
1,0.312536,-0.609284,1.804243,-1.042158,0.684861,-0.367509,1.381386,-1.672624,-1.192286,-0.529291,1
2,-2.117198,3.922800,-2.100930,-1.534785,0.170328,-1.863566,-2.318735,1.381744,-1.589880,-1.385899,0
3,-0.136663,-0.464429,2.460730,-3.415477,-1.199578,-1.955750,1.633197,-0.840673,-1.115010,0.109400,0
4,-5.020961,6.298293,-3.202072,-2.571758,0.475749,-2.899967,-2.939422,4.970843,-1.174923,-0.999756,0


In [3]:
#Crear columna ID
if 'ID_cliente' not in df.columns:
    df.insert(0, 'ID_Cliente', [f"C{str(i).zfill(6)}" for i in range(1, len(df)+1)])

df.head

<bound method NDFrame.head of        ID_Cliente  feature_1  feature_2  feature_3  feature_4  feature_5  \
0         C000001  -0.623609   0.373243   0.151976  -0.860282   0.009626   
1         C000002   0.312536  -0.609284   1.804243  -1.042158   0.684861   
2         C000003  -2.117198   3.922800  -2.100930  -1.534785   0.170328   
3         C000004  -0.136663  -0.464429   2.460730  -3.415477  -1.199578   
4         C000005  -5.020961   6.298293  -3.202072  -2.571758   0.475749   
...           ...        ...        ...        ...        ...        ...   
499995    C499996  -1.198644   2.156196   1.637884  -0.190452  -1.551630   
499996    C499997  -0.134910   0.599102   0.679365   1.070906  -1.690005   
499997    C499998  -1.718049   3.768727  -2.118315   0.585613  -1.187085   
499998    C499999  -2.275005   5.075794  -3.593994   2.000689   1.113220   
499999    C500000  -0.202541   1.002527  -2.049604   1.328608  -1.319736   

        feature_6  feature_7  feature_8  feature_9  featu

In [4]:
#Monto de crédito

df['monto_credito'] = (
    df['feature_1'] * 1000+
    df['feature_2'] * 200 +
    np.random.normal(0, 500, size=len(df))
)

df[['feature_1', 'feature_2', 'monto_credito']].head

<bound method NDFrame.head of         feature_1  feature_2  monto_credito
0       -0.623609   0.373243    -154.224663
1        0.312536  -0.609284    -664.272402
2       -2.117198   3.922800    -580.070948
3       -0.136663  -0.464429    -417.284341
4       -5.020961   6.298293   -3639.939409
...           ...        ...            ...
499995  -1.198644   2.156196    -729.049330
499996  -0.134910   0.599102     798.093927
499997  -1.718049   3.768727    -909.120586
499998  -2.275005   5.075794   -2253.076532
499999  -0.202541   1.002527    -432.221218

[500000 rows x 3 columns]>

In [5]:
X = df[['feature_1']]
y = df[['monto_credito']]

In [6]:
print(df.columns)

Index(['ID_Cliente', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'moroso', 'monto_credito'],
      dtype='object')


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state=42
)

# Ids de los casos de test
id_test = df.loc[y_test.index, 'ID_Cliente'].reset_index(drop=True)

X_train.shape, X_test.shape

((350000, 1), (150000, 1))

In [8]:
#Entrenando el modelo
reg = LinearRegression()
reg.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [9]:
y_pred = reg.predict(X_test)
y_pred[:5]

array([[  727.96487039],
       [-1118.55674301],
       [ -245.44137638],
       [-2221.88577298],
       [ 1021.75898199]])

In [10]:
#Evaluar modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R2:", r2)

MSE: 359671.57451894664
R2: 0.8613894099783002


In [14]:
print("id_test:", id_test.shape, type(id_test))
print("y_test_reset:", y_test_reset.shape, type(y_test_reset))
print("y_pred_series:", y_pred_series.shape, type(y_pred_series))


id_test: (150000,) <class 'pandas.core.series.Series'>
y_test_reset: (150000, 1) <class 'pandas.core.frame.DataFrame'>
y_pred_series: (150000,) <class 'pandas.core.series.Series'>


In [15]:
# Aplanar el array a 1D
y_pred = y_pred.ravel()

# Convertir y_pred a Series
y_pred_series = pd.Series(y_pred)

# Reiniciar índices de y_test
y_test_reset = y_test.reset_index(drop=True)
y_test_reset = y_test_reset.squeeze()

# Creando DF de resultados
df_resultados = pd.DataFrame({
    'ID_Cliente': id_test,
    'monto_credito_real': y_test_reset,
    'monto_credito_predicho': y_pred_series,
    'error_absoluto': np.abs(y_test_reset - y_pred_series)
})

df_resultados.head()


,ID_Cliente,monto_credito_real,monto_credito_predicho,error_absoluto
0,C104242,1024.127515,727.964870,296.162645
1,C199677,-880.042314,-1118.556743,238.514429
2,C140200,128.662180,-245.441376,374.103557
3,C132815,-2600.635635,-2221.885773,378.749862
4,C408698,974.636302,1021.758982,47.122680


In [16]:
resultados_dir = os.path.join(os.getcwd(),'..','resultado de modelos')
os.makedirs(resultados_dir, exist_ok=True)

#Nombre del archivo
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"RegresionLineal_resultados_{timestamp}.csv"
output_path = os.path.join(resultados_dir, output_file)

#Guardar
df_resultados.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"Archivo de resultados guardado en: {output_path}")

Archivo de resultados guardado en: d:\IC\Y03AN1_Machine_Learning_I\semana1\..\resultado de modelos\RegresionLineal_resultados_20250703_213117.csv
